In [8]:
from datetime import datetime, timedelta
import json
import requests
import time

BASE_URL = "https://api.weather.gov"
RATE_LIMIT_SECONDS = 1   # prevent the API from getting mad for requesting too much data at once

def print_readable_json(json_string):
    readable_json = json.dumps(json_string, indent=4, sort_keys=True)
    print(readable_json)

# Get all stations in GA

Print the first 10

In [12]:
response = requests.get(f"{BASE_URL}/stations?state=GA")
stations = response.json()["features"][0:10]
print_readable_json(stations)

[
    {
        "geometry": {
            "coordinates": [
                -84.39281,
                33.77232
            ],
            "type": "Point"
        },
        "id": "https://api.weather.gov/stations/0349W",
        "properties": {
            "@id": "https://api.weather.gov/stations/0349W",
            "@type": "wx:ObservationStation",
            "county": "https://api.weather.gov/zones/county/GAC121",
            "elevation": {
                "unitCode": "wmoUnit:m",
                "value": 291.9984
            },
            "fireWeatherZone": "https://api.weather.gov/zones/fire/GAZ033",
            "forecast": "https://api.weather.gov/zones/forecast/GAZ033",
            "name": "Georgia Tech Bobby Dodd Stadium",
            "stationIdentifier": "0349W",
            "timeZone": "America/New_York"
        },
        "type": "Feature"
    },
    {
        "geometry": {
            "coordinates": [
                -84.40367,
                33.77494
            ],
     

# Show metadata about CRC observation station

In [14]:
response = requests.get(f"{BASE_URL}/stations/0353W")
print_readable_json(response.json()["properties"])

{
    "@id": "https://api.weather.gov/stations/0353W",
    "@type": "wx:ObservationStation",
    "county": "https://api.weather.gov/zones/county/GAC121",
    "elevation": {
        "unitCode": "wmoUnit:m",
        "value": 305.1048
    },
    "fireWeatherZone": "https://api.weather.gov/zones/fire/GAZ033",
    "forecast": "https://api.weather.gov/zones/forecast/GAZ033",
    "name": "Georgia Tech Campus Recreation Center",
    "stationIdentifier": "0353W",
    "timeZone": "America/New_York"
}


# Show latest observations at CRC observation station

In [16]:
response = requests.get(f"{BASE_URL}/stations/0353W/observations/latest")
print_readable_json(response.json()["properties"])

{
    "@id": "https://api.weather.gov/stations/0353W/observations/2024-01-17T22:20:00+00:00",
    "@type": "wx:ObservationStation",
    "barometricPressure": {
        "qualityControl": "V",
        "unitCode": "wmoUnit:Pa",
        "value": 102336.62
    },
    "cloudLayers": [],
    "dewpoint": {
        "qualityControl": "V",
        "unitCode": "wmoUnit:degC",
        "value": -13.89
    },
    "elevation": {
        "unitCode": "wmoUnit:m",
        "value": 305.1
    },
    "heatIndex": {
        "qualityControl": "V",
        "unitCode": "wmoUnit:degC",
        "value": null
    },
    "icon": null,
    "maxTemperatureLast24Hours": {
        "unitCode": "wmoUnit:degC",
        "value": null
    },
    "minTemperatureLast24Hours": {
        "unitCode": "wmoUnit:degC",
        "value": null
    },
    "precipitationLast3Hours": {
        "qualityControl": "Z",
        "unitCode": "wmoUnit:mm",
        "value": null
    },
    "presentWeather": [],
    "rawMessage": "",
    "relativ

# Get barometric pressure at observation station for specific days

Observations appear to be hourly.  Timestamp in the route parameters is an ISO-8601 string, with no fractional seconds

In [5]:
response = requests.get(f"{BASE_URL}/stations/0353W/observations/2024-01-16T13:00:00Z")
print_readable_json(response.json())

{
    "@context": [
        "https://geojson.org/geojson-ld/geojson-context.jsonld",
        {
            "@version": "1.1",
            "@vocab": "https://api.weather.gov/ontology#",
            "bearing": {
                "@type": "s:QuantitativeValue"
            },
            "city": "s:addressLocality",
            "county": {
                "@type": "@id"
            },
            "distance": {
                "@id": "s:Distance",
                "@type": "s:QuantitativeValue"
            },
            "forecastGridData": {
                "@type": "@id"
            },
            "forecastOffice": {
                "@type": "@id"
            },
            "geo": "http://www.opengis.net/ont/geosparql#",
            "geometry": {
                "@id": "s:GeoCoordinates",
                "@type": "geo:wktLiteral"
            },
            "publicZone": {
                "@type": "@id"
            },
            "s": "https://schema.org/",
            "state": "s:addressReg

In [11]:
def format_timestamp(raw_datetime):
    # NOAA API only records observations at hourly intervals with a Zulu UTC offset
    return raw_datetime.strftime("%Y-%m-%dT%H:00:00Z")

def request_barometric_pressure(station, timestamp):
    response = requests.get(f"{BASE_URL}/stations/{station}/observations/{timestamp}")
    time.sleep(RATE_LIMIT_SECONDS)
    if response.status_code == 404:
        print(f"Data not found for date: {timestamp}")
        return ()
    else:
        pressure_data = response.json()["properties"]["barometricPressure"]
        return (pressure_data["qualityControl"], pressure_data["unitCode"], pressure_data["value"])

STATION_CODE = "0353W"  # CRC station
HOUR_CONSTANT = 13      # getting pressure at a fixed hour every day (8 or 9AM Eastern depending on whether DST is on)
now = datetime.now()
start_datetime = now.replace(hour=HOUR_CONSTANT)
timestamps = [format_timestamp(start_datetime - timedelta(days=i)) for i in range(10)]
pressures = [request_barometric_pressure(STATION_CODE, time) for time in timestamps]
print(pressures)

Data not found for date: 2024-01-10T13:00:00Z
Data not found for date: 2024-01-09T13:00:00Z
Data not found for date: 2024-01-08T13:00:00Z
[('V', 'wmoUnit:Pa', 102404.34), ('Z', 'wmoUnit:Pa', None), ('Z', 'wmoUnit:Pa', None), ('Z', 'wmoUnit:Pa', None), ('Z', 'wmoUnit:Pa', None), ('Z', 'wmoUnit:Pa', None), ('Z', 'wmoUnit:Pa', None), (), (), ()]


Looks like data is only kept for around a week.  Also, the "quality control" code `Z` might have some impact as to whether there is data or not.